In [1]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import numpy

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
df = pd.read_csv('/kaggle/input/tesla-stock-price/Tesla.csv - Tesla.csv.csv')
df1= df['Close']
df1.shape

In [5]:
import matplotlib.pyplot as plt
plt.plot(df1)

In [6]:
# Scaling the data in the range of (-1,1) using minmax scaler

scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

print(df1)

In [8]:
training_size= int(len(df1)*.70)
test_size= len(df1)-training_size

train_data, test_data = df1[0:training_size,:],df1[training_size:len(df1),:1]

In [9]:
print(training_size,test_size)
print(test_data.shape)
train_data.shape

In [10]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]    
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [11]:
time_step=100
x_train, y_train =create_dataset(train_data, time_step)
x_test, y_test = create_dataset(test_data, time_step)

In [12]:
print(x_test.shape),print(y_test.shape)
print(x_train.shape), print(y_train.shape)

In [13]:
# Reshaping the train and test data for input to LSTM.

x_train= x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test= x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

print(x_train.shape)
x_test.shape

In [14]:
# Building an LSTM Model using tensorflow keras sequential API

lstm_model= Sequential()
lstm_model.add(LSTM(50,return_sequences= True,input_shape = (100,1)))
lstm_model.add(LSTM(50,return_sequences= True))
lstm_model.add(LSTM(50,return_sequences= True))
lstm_model.add(Dense(1))               

In [15]:
# Compiling the LSTM model

lstm_model.compile(loss='mean_squared_error',optimizer='adam')

In [16]:
lstm_model.fit(x_train, y_train , validation_data= (x_test,y_test), epochs=100 , batch_size=64, verbose=1)

In [17]:
# prediction using test data.
train_predict=lstm_model.predict(x_train)
test_predict=lstm_model.predict(x_test)


In [18]:
test_predict.shape

In [19]:
train_predict= train_predict.reshape(1083,100)
test_predict= test_predict.reshape(407,100)

In [20]:
# transform back the predicted value since the predicted values are in range (-1,1)
train_predict = scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
# plotting the results


look_back=100
trainPredictPlot = numpy.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

# shift test predictions for plotting
testPredictPlot = numpy.empty_like(df1)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(df1))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()